In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Lambda
import torchinfo

import numpy as np
import math
import time

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU Device:【{}:{}】".format(device.type, device.index))
    torch.cuda.set_device(0)

GPU Device:【cuda:None】


In [3]:
class CustomDataset(Dataset):
    def __init__(self, data:np.ndarray, labels:np.ndarray, transform=ToTensor(), 
    target_transform=Lambda(lambda y: torch.zeros(2, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))):
        self.data:torch.Tensor = torch.from_numpy(np.swapaxes(data, 1, 2))
        self.labels:torch.Tensor = torch.from_numpy(labels)
        self.transform = None
        self.target_transform = None
        # self.transform = transform
        # self.target_transform = target_transform
        # self.shuffle()
    
    def shuffle(self, seed=None):
        '\n        seed(self, seed=None)\n\n        Reseed a legacy MT19937 BitGenerator\n        '
        self.shuffle_seed = np.random.randint(1, 65535) if seed is None else seed
        print(f"随机种子：{self.shuffle_seed}")
        np.random.seed(self.shuffle_seed)
        np.random.shuffle(self.data)
        np.random.seed(self.shuffle_seed)
        np.random.shuffle(self.labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx, 0]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

In [4]:
def load_dataset(path="dataset.npz", train_percent=0.8) -> tuple:
    with np.load(path) as dataset:
        full_data = dataset["data"].astype(np.float32)
        full_labels = dataset["labels"].astype(np.int64)
    train_size = int(full_data.shape[0]*train_percent)
    test_size = full_data.shape[0]-train_size
    seed = np.random.randint(1, 65535) # 35468
    np.random.seed(seed)
    np.random.shuffle(full_data)
    np.random.seed(seed)
    np.random.shuffle(full_labels)
    train_data, test_data = full_data[:train_size], full_data[train_size:]
    train_labels, test_labels = full_labels[:train_size], full_labels[train_size:]
    print(f"训练集大小：{train_size}", f"测试集大小：{test_size}", f"随机种子：{seed}")
    train_dataset = CustomDataset(train_data, train_labels)
    test_dataset = CustomDataset(test_data, test_labels)
    return train_dataset, test_dataset

In [5]:
train_dataset, test_dataset = load_dataset("D:\\datasets\\ABIDE\\ABIDE_augmented_dataset.npz", 0.8)

训练集大小：9636 测试集大小：2409 随机种子：12160


In [6]:
batch_size=64

# Create data loaders.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for X, y in test_dataloader:
    data_shape = X.shape
    label_shape = y.shape
    print(f"Shape of X [N, L, H]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, L, H]: torch.Size([64, 116, 60])
Shape of y: torch.Size([64]) torch.int64


In [7]:
class DWConv(nn.Module):
    def __init__(self, dim=768):
        super(DWConv, self).__init__()
        self.dwconv = nn.Conv1d(dim, dim, 3, 1, 1, bias=True, groups=dim)

    def forward(self, x):
        x = self.dwconv(x)
        return x

In [8]:
class LKA(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.conv0 = nn.Conv1d(dim, dim, 5, padding=2, groups=dim)
        self.conv_spatial = nn.Conv1d(dim, dim, 7, stride=1, padding=9, groups=dim, dilation=3)
        self.conv1 = nn.Conv1d(dim, dim, 1)

    def forward(self, x):
        u = x.clone()        
        attn = self.conv0(x)
        attn = self.conv_spatial(attn)
        attn = self.conv1(attn)

        return u * attn

In [9]:
class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Conv1d(in_features, hidden_features, 1)
        self.dwconv = DWConv(hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Conv1d(hidden_features, out_features, 1)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.dwconv(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

In [10]:
class Attention(nn.Module):
    def __init__(self, d_model):
        super().__init__()

        self.proj_1 = nn.Conv1d(d_model, d_model, 1)
        self.activation = nn.GELU()
        self.spatial_gating_unit = LKA(d_model)
        self.proj_2 = nn.Conv1d(d_model, d_model, 1)

    def forward(self, x):
        shorcut = x.clone()
        x = self.proj_1(x)
        x = self.activation(x)
        x = self.spatial_gating_unit(x)
        x = self.proj_2(x)
        x = x + shorcut
        return x

In [11]:
class Block(nn.Module):
    def __init__(self, dim, mlp_ratio=4., drop=0., act_layer=nn.GELU):
        super().__init__()
        self.norm1 = nn.BatchNorm1d(dim)
        self.attn = Attention(dim)

        self.norm2 = nn.BatchNorm1d(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x + self.attn(x)
        x = x + self.mlp(x)
        # x = x + self.attn(self.norm1(x))
        # x = x + self.mlp(self.norm2(x))
        return x

In [12]:
class OverlapPatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=7, stride=4, in_chans=3, embed_dim=768):
        super().__init__()
        self.proj = nn.Conv1d(in_chans, embed_dim, kernel_size=patch_size, stride=stride,
                              padding=patch_size//2)
        self.norm = nn.BatchNorm1d(embed_dim)

    def forward(self, x):
        x = self.proj(x)
        _, L, H = x.shape
        # x = self.norm(x)
        return x, L, H

In [13]:
class VAN(nn.Module):
    def __init__(self, img_size=224, in_chans=3, num_classes=1000, embed_dims=[64, 128, 256, 512],
                mlp_ratios=[4, 4, 4, 4], drop_rate=0., norm_layer=nn.LayerNorm,
                 depths=[3, 4, 6, 3], num_stages=4, device=torch.device("cuda")):
        super().__init__()
        self.num_classes = num_classes
        self.depths = depths
        self.num_stages = num_stages

        for i in range(num_stages):
            patch_embed = OverlapPatchEmbed(img_size=img_size if i == 0 else img_size // (2 ** (i + 1)),
                                            patch_size=7 if i == 0 else 3,
                                            stride=4 if i == 0 else 2,
                                            in_chans=in_chans if i == 0 else embed_dims[i - 1],
                                            embed_dim=embed_dims[i])

            block = nn.ModuleList([Block(dim=embed_dims[i], mlp_ratio=mlp_ratios[i], drop=drop_rate)
                for j in range(depths[i])])
            norm = norm_layer(embed_dims[i])

            setattr(self, f"patch_embed{i + 1}", patch_embed)
            setattr(self, f"block{i + 1}", block)
            setattr(self, f"norm{i + 1}", norm)

        # classification head
        self.head = nn.Linear(embed_dims[3], num_classes) if num_classes > 0 else nn.Identity()

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv1d):
            fan_out = m.kernel_size[0] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward_features(self, x):
        B = x.shape[0]

        for i in range(self.num_stages):
            patch_embed = getattr(self, f"patch_embed{i + 1}")
            block = getattr(self, f"block{i + 1}")
            norm = getattr(self, f"norm{i + 1}")
            x, L, H = patch_embed(x)
            for blk in block:
                x = blk(x)
            x = x.flatten(2).transpose(1, 2)
            x = norm(x)
            if i != self.num_stages - 1:
                x = x.reshape(B, H, -1).permute(0, 2, 1).contiguous()

        return x.mean(dim=1)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)

        return x


In [14]:
class MyVAN(nn.Module):
    def __init__(self, img_size=224, in_chans=3, num_classes=1000, embed_dims=[64, 128, 256, 512],
                mlp_ratios=[4, 4, 4, 4], drop_rate=0., norm_layer=nn.LayerNorm,
                 depths=[3, 4, 6, 3], num_stages=4, device=torch.device("cuda")):
        super().__init__()
        self.num_classes = num_classes
        self.depths = depths
        self.num_stages = num_stages

        self.device:torch.DeviceObjType = device
        # self.cache:bool = cache
        # self.__cache:list(torch.TensorType) = []
        # self.copy_block:bool = copy_block
        
        self._embed_dims = embed_dims
        self._mlp_ratios = mlp_ratios
        self.drop_rate = drop_rate

        for i in range(num_stages):
            patch_embed = OverlapPatchEmbed(img_size=img_size if i == 0 else img_size // (2 ** (i + 1)),
                                            patch_size=7 if i == 0 else 3,
                                            stride=4 if i == 0 else 2,
                                            in_chans=in_chans if i == 0 else embed_dims[i - 1],
                                            embed_dim=embed_dims[i])

            block = nn.ModuleList([Block(dim=embed_dims[i], mlp_ratio=mlp_ratios[i], drop=drop_rate)])
            norm = norm_layer(embed_dims[i])

            setattr(self, f"patch_embed{i + 1}", patch_embed)
            setattr(self, f"block{i + 1}", block)
            setattr(self, f"norm{i + 1}", norm)

        # classification head
        self.head = nn.Linear(embed_dims[3], num_classes) if num_classes > 0 else nn.Identity()

        self.apply(self._init_weights)
        self._block_full = False

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)
        elif isinstance(m, nn.Conv1d):
            fan_out = m.kernel_size[0] * m.out_channels
            fan_out //= m.groups
            m.weight.data.normal_(0, math.sqrt(2.0 / fan_out))
            if m.bias is not None:
                m.bias.data.zero_()

    def forward_features(self, x):
        # B = x.shape[0]

        for i in range(self.num_stages):
            patch_embed = getattr(self, f"patch_embed{i + 1}")
            block = getattr(self, f"block{i + 1}")
            norm = getattr(self, f"norm{i + 1}")
            x, L, H = patch_embed(x)
            for blk in block:
                x = blk(x)
            x = x.transpose(1, 2)
            x = norm(x)
            if i != self.num_stages - 1:
                x = x.transpose(2, 1)

        return x.mean(dim=1)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)

        return x

    def freeze(self, block:Block):
        for param in block.parameters():
            param.requires_grad = False

    # def __forward_cache(self, block):
    #     if self.cache:
    #         for batch, X in enumerate(self.__cache):
    #             self.__cache[batch] = block(X.to(self.device))

    def addNewBlock(self, copy_last=False):
        if self._block_full:
            return
        print("----------")
        print(f"{'Copying' if copy_last else 'Adding'} new block...")
        cnt = 0
        for i in range(self.num_stages):
            blocks:nn.ModuleList = getattr(self, f"block{i + 1}")
            if len(blocks) >= self.depths[i]:
                continue
            newBlock = Block(self._embed_dims[i], self._mlp_ratios[i], self.drop_rate)
            last_block:Block = blocks[-1]
            if copy_last:
                newBlock.load_state_dict(last_block.state_dict())
            blocks.append(newBlock)
            cnt += 1
            self.freeze(last_block)
            newBlock.to(self.device)
        # self.__forward_cache(last_block)
        if cnt == 0:
            self._block_full = True
        print("Success!" if cnt else "Full!")

In [15]:
def test(model:nn.Module, dataloader:DataLoader, loss_fn, device=torch.device("cuda")):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    # model.train()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model.forward(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"-- Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
def train(model:nn.Module, train_dataloader:DataLoader, loss_fn, optimizer,
    batch_size:int, epochs:int, add_new_block=False, copy_block=False, use_cache=False, device=torch.device("cuda"), test_dataloader:DataLoader=None):
    for X, y in train_dataloader:
        data_shape = X.shape
        label_shape = y.shape
        break
    size = len(train_dataloader.dataset)
    num_batches = len(train_dataloader)
    for epoch in range(epochs):
        model.train()
        print(f"Epoch: {epoch+1}/{epochs}")
        if add_new_block and epoch and epoch%5 == 0:
            model.addNewBlock(copy_block)
        loss, correct = 0, 0
        torch.cuda.synchronize()
        time_delta = 0
        for batch, (X, y) in enumerate(train_dataloader):
            time_start = time.time()
            if use_cache:
                if epoch == 0:
                    model.__cache.append(X)
                X = model.__cache[batch].to(device)
            else:
                X = X.to(device)
            y = y.to(device)

            # Compute prediction error
            pred = model.forward(X)
            batch_loss = loss_fn(pred, y)

            # Backpropagation
            batch_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            current = batch * batch_size + len(X)

            batch_loss = batch_loss.item()
            loss += batch_loss
            batch_correct = (pred.argmax(1) == y).type(torch.float).sum().item()
            correct += batch_correct

            batch_correct /= len(X)
            # time_batch_end = time.time()
            torch.cuda.synchronize()
            time_end = time.time()
            time_delta += (time_end - time_start)
            print(f"\r{batch+1}/{num_batches}  [{current:>3d}/{size:>3d}] - batch loss: {batch_loss:>7f} - batch accuracy: {(100*batch_correct):>0.1f}% - {time_delta/(batch+1)*1000:>0.3f}ms/batch", end = "", flush=True)
        loss /= num_batches
        correct /= size
        torch.cuda.synchronize()
        time_end = time.time()
        print(f"\n-- Average loss: {loss:>7f} - Accuracy: {(100*correct):>0.1f}% - {time_delta*1000:>0.3f}ms")
        if test_dataloader is not None:
            test(model, test_dataloader, loss_fn, device)
    print("\n", torchinfo.summary(model, input_size=data_shape))

In [17]:
lr = 1e-4
epochs = 5
# loss_fn = nn.CrossEntropyLoss
# optimizer = torch.optim.Adam

In [18]:
model = VAN(data_shape[2], data_shape[1], 2, device=device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
model.to(device)
train(model, train_dataloader, loss_fn, optimizer, batch_size, epochs, device=device, test_dataloader=test_dataloader)

Epoch: 1/5
151/151  [9636/9636] - batch loss:     nan - batch accuracy: 44.4% - 125.596ms/batch
-- Average loss:     nan - Accuracy: 45.9% - 18964.998ms
-- Test Error: 
 Accuracy: 46.0%, Avg loss:      nan 

Epoch: 2/5
151/151  [9636/9636] - batch loss:     nan - batch accuracy: 44.4% - 87.690ms/batch
-- Average loss:     nan - Accuracy: 45.9% - 13241.205ms
-- Test Error: 
 Accuracy: 46.0%, Avg loss:      nan 

Epoch: 3/5
151/151  [9636/9636] - batch loss:     nan - batch accuracy: 44.4% - 85.251ms/batch
-- Average loss:     nan - Accuracy: 45.9% - 12872.850ms
-- Test Error: 
 Accuracy: 46.0%, Avg loss:      nan 

Epoch: 4/5
151/151  [9636/9636] - batch loss:     nan - batch accuracy: 44.4% - 85.495ms/batch
-- Average loss:     nan - Accuracy: 45.9% - 12909.799ms
-- Test Error: 
 Accuracy: 46.0%, Avg loss:      nan 

Epoch: 5/5
151/151  [9636/9636] - batch loss:     nan - batch accuracy: 44.4% - 91.233ms/batch
-- Average loss:     nan - Accuracy: 45.9% - 13776.193ms
-- Test Error: 
 Ac

In [19]:
my_model = MyVAN(data_shape[2], data_shape[1], 2, device=device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, my_model.parameters()), lr=lr)
my_model.to(device)
train(my_model, train_dataloader, loss_fn, optimizer, batch_size, epochs, add_new_block=True, copy_block=True, device=device, test_dataloader=test_dataloader)

Epoch: 1/5
151/151  [9636/9636] - batch loss: 0.689021 - batch accuracy: 44.4% - 30.926ms/batch
-- Average loss: 0.706649 - Accuracy: 51.9% - 4669.848ms
-- Test Error: 
 Accuracy: 50.5%, Avg loss: 0.695382 

Epoch: 2/5
151/151  [9636/9636] - batch loss: 0.542509 - batch accuracy: 72.2% - 30.747ms/batch
-- Average loss: 0.621162 - Accuracy: 65.7% - 4642.832ms
-- Test Error: 
 Accuracy: 49.9%, Avg loss: 0.747441 

Epoch: 3/5
151/151  [9636/9636] - batch loss: 0.113019 - batch accuracy: 97.2% - 31.039ms/batch
-- Average loss: 0.302967 - Accuracy: 88.1% - 4686.888ms
-- Test Error: 
 Accuracy: 50.4%, Avg loss: 1.130643 

Epoch: 4/5
151/151  [9636/9636] - batch loss: 0.052794 - batch accuracy: 97.2% - 29.460ms/batchh
-- Average loss: 0.157281 - Accuracy: 93.8% - 4448.464ms
-- Test Error: 
 Accuracy: 49.9%, Avg loss: 1.350594 

Epoch: 5/5
151/151  [9636/9636] - batch loss: 0.045421 - batch accuracy: 100.0% - 29.060ms/batch
-- Average loss: 0.152261 - Accuracy: 93.8% - 4388.016ms
-- Test Error

In [20]:
test(model, train_dataloader, loss_fn, device=device)
test(my_model, train_dataloader, loss_fn, device=device)

test(model, test_dataloader, loss_fn, device=device)
test(my_model, test_dataloader, loss_fn, device=device)

-- Test Error: 
 Accuracy: 45.9%, Avg loss:      nan 

-- Test Error: 
 Accuracy: 98.2%, Avg loss: 0.056814 

-- Test Error: 
 Accuracy: 46.0%, Avg loss:      nan 

-- Test Error: 
 Accuracy: 52.2%, Avg loss: 1.509564 

